In [1]:
import pandas as pd 
import numpy as np 
import infrastructure
import filepath 
import pygsheets
import datetime as dt
import openpyxl 
import os
from pathlib import Path

In [2]:
import xlsxwriter
import send_email
from colorama import Fore, Style

# IMPORT DATA

In [3]:
#manual input
df = pd.read_csv('SS_LC_merged_data.csv')

/Users/nathan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1,2,3,4,6,8,9,10,11,12,14,16,25,26,27,28,29,30,31,32,33,34,35,36,43,47,50,53,56,57) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# SMS testing pipeline
sms_test = infrastructure.get_smartsheet('test_sms')
sms_test['Hitpath Offer ID'] = sms_test['Hitpath Offer ID'].astype(str)
# SMS offer sheet 
sms_offer = infrastructure.get_smartsheet('offers_sms')
sms_offer = sms_offer[sms_offer['Hitpath Offer ID'].isna() == False]
# sms_offer = sms_offer[sms_offer['Hitpath ID'].str.contains("TEST") == False]
# sms_offer = sms_offer[~sms_offer['Hitpath ID'].str.contains(r'test\d*', regex=True)]
sms_offer['Hitpath Offer ID'] = sms_offer['Hitpath Offer ID'].astype(int).astype(str)
# Email offer sheet 
email_offer = infrastructure.get_smartsheet('offers_email')
# get mamba for shortcode 
mamba = infrastructure.get_mamba()
# Offer Testing Timeline 
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
timeline = gc.open_by_url('https://docs.google.com/spreadsheets/d/16vrHMWs0ambcBJ1sC0SqpYVG8SOSWlbg1N22-bF49v8/edit#gid=423634805') 
timeline_wks  = timeline[1].get_as_df()

In [5]:
#Publisher Config
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
publisher_config = gc.open_by_url(
    'https://docs.google.com/spreadsheets/d/1Tzda6Djr3zQmOhWu7Ief3GVR9Cjaml8238CeX7chj_U/edit#gid=1620368362') 
pub_config = publisher_config[0].get_as_df()
pub_config.rename(columns={'DP.DS or DP.sV': 'DP.SV'}, inplace=True)

/Users/nathan/opt/anaconda3/lib/python3.8/site-packages/pygsheets/worksheet.py:1554: UserWarning: At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.
  warnings.warn('At least one column name in the data frame is an empty string. If this is a concern, please specify include_tailing_empty=False and/or ensure that each column containing data has a name.')


In [6]:
#la nina
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
lanina = gc.open_by_url('https://docs.google.com/spreadsheets/d/1obszkCQoE0ELOR1O0CrLVETUEmEIWlGuyAmK3FgWSJg/edit#gid=1099746391') 
lanina_wks  = lanina[1].get_as_df()

In [7]:
#sms offer restrictions
sms_restrictions = sms_offer[['Hitpath Offer ID', 'Paused Pubs']].copy()

In [8]:
#rename and retype
df.rename(columns={'Hitpath_Offer_ID': 'Hitpath Offer ID'}, inplace=True)
df['Hitpath Offer ID'] = pd.to_numeric(df['Hitpath Offer ID'], errors='coerce')
df = df.dropna(subset=['Hitpath Offer ID'])
df['Hitpath Offer ID'] = df['Hitpath Offer ID'].astype(int).astype(str)

df['Affiliate_Id'] = pd.to_numeric(df['Affiliate_Id'], errors='coerce')
df = df.dropna(subset=['Affiliate_Id'])
df['Affiliate_Id'] = df['Affiliate_Id'].astype(int).astype(str)


In [9]:
df = df[df['Send Strategy'] != 'AR']

In [10]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [11]:
#mamba schedule from today and onward
today = pd.to_datetime(dt.date.today())

mamba_future = mamba[mamba['Date'] >= today]

mamba_future

,Date,Dataset,Drop,Time,Segment,Send Strategy,Offer,Limit,Offset,Creative,Job Name,Hitpath Offer ID,Shortcode,shortcode_DP.SV
86804,2023-12-20,DSS_RHD.CC_461263,Drop 1,12:00 PM PST,DSS_RHD.CC_30DC_TMO,P,11646 - B2 CGC DSS,,,CGC.11646.SC.DSS.451492,SS_DSS_RHD-CC-30DC-TMO_11646_P_20Dec23,11646.0,DSS,DSS_RHD.CC
86810,2023-12-20,DSS_ZM.PL.2_461681,Drop 1,12:00 PM PST,DSS_ZM.PL.2_30DC,P,11928 - AM LS DSS,,,LS.11928.SC.DSS.451552,SS_DSS_ZM-PL-2-30DC_11928_P_20Dec23,11928.0,DSS,DSS_ZM.PL.2
86816,2023-12-20,FLC_AI.CC_460939,Drop 1,12:00 PM PST,FLC_AI.CC_30DC,P,12089 - BE DS FLC,,,DS.12089.SC.FLC.451322,SS_FLC_AI-CC-30DC_12089_P_20Dec23,12089.0,FLC,FLC_AI.CC
86822,2023-12-20,FLC_EDM.247L_461227,Drop 1,12:00 PM PST,FLC_EDM.247L_30DC_NEXL,P,12610 - AVL CL FLC,,,CL.12610.SC.FLC.452321,SS_FLC_EDM-247L-30DC-NEXL_12610_P_20Dec23,12610.0,FLC,FLC_EDM.247L
86824,2023-12-20,FLC_EDM.247L_461227,Drop 3,12:00 PM PST,FLC_EDM.247L_30DC_EXL,P,12610 - AVL CL FLC,,,CL.12610.SC.FLC.452321,SS_FLC_EDM-247L-30DC-EXL_12610_P_20Dec23,12610.0,FLC,FLC_EDM.247L
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100845,2024-01-30,SVT_AL.PL_461794,Drop 1,10:00 AM PST,SVT_AL.PL_30DC,P,11600 - WI UG SVT,,,UG.11600.SC.SVT.452091,SS_SVT_AL-PL-30DC_11600_P_30Jan24,11600.0,SVT,SVT_AL.PL
100852,2024-01-30,UAA_JET.ZTA_461835,Drop 1,10:00 AM PST,UAA_JET.ZTA_30DC,P,10086 - BB NP UAA,,,NP.10086.SC.UAA.452391,SS_UAA_JET-ZTA-30DC_10086_P_17Dec23,10086.0,UAA,UAA_JET.ZTA
100874,2024-01-30,MBC_AI.CC_460939,Drop 1,10:00 AM PST,MBC_AI.CC_30DC_TMO,P,8838 - W4 EDU MBC,,,EDU.8838.SC.MBC.450832,SS_MBC_AI-CC-30DC-TMO_8838_P_30Jan24,8838.0,MBC,MBC_AI.CC
100904,2024-01-30,MBC_SPK.CR_460921,Drop 1,10:00 AM PST,MBC_SPK.CR_30DC_TMO,P,11468 - SM RR MBC,,,RR.11468.SC.MBC.451052,SS_MBC_SPK-CR-30DC-TMO_11468_P_30Jan24,11468.0,MBC,MBC_SPK.CR


# Most Recent Passed Offers

In [12]:
sms_offer['Scheduled Test Date'] = pd.to_datetime(sms_offer['Scheduled Test Date'], errors='coerce')

#get the current date to calculate the last 30 days range
current_date = pd.to_datetime('now').normalize()

sms_offer_filtered = sms_offer[
    (sms_offer['Status'] == 'Live') &
    (sms_offer['Scheduled Test Date'] >= current_date - pd.Timedelta(days=30))
]

sms_offer_filtered.reset_index(drop=True, inplace=True)
sms_offer_filtered = sms_offer_filtered[
    ['Hitpath Offer ID', 'Scheduling Name', 'Advertiser', '$ Payout', 'Payout Type', 'Vertical', 'Day Restrictions']
]

In [13]:
#calculating eCPM for passed offers from PT performance
df_pt = df[df['Send Strategy'] == 'PT']
merged_df = pd.merge(sms_offer_filtered, df_pt, on='Hitpath Offer ID', how='left')

# Grouping by 'Hitpath ID' and calculating total revenue and total delivered for each
grouped_stats = merged_df.groupby('Hitpath Offer ID').agg({'Revenue': 'sum', 'Delivered': 'sum'}).reset_index()
passed_offers = pd.merge(sms_offer_filtered, grouped_stats, on='Hitpath Offer ID', how='left')
passed_offers['eCPM'] = passed_offers['Revenue']/passed_offers['Delivered']*1000

In [14]:
#vertical rec
df_vertical = df.merge(pub_config[['DP.SV', 'Sub Vertical', 'INTERNAL STATUS']], on='DP.SV', how='left')
df_vertical = df_vertical.merge(sms_offer[['Hitpath Offer ID', 'Vertical', 'Paused Pubs', 'Status']], on='Hitpath Offer ID', how='left')

In [15]:
grouped_data = df_vertical.groupby(['Vertical', 'Sub Vertical']).agg({'Revenue': 'sum', 'Delivered': 'sum'}).reset_index()

# eCPM for grouped data
grouped_data['eCPM'] = (grouped_data['Revenue']/grouped_data['Delivered'])*1000

# Filtering for combinations where the calculated eCPM is greater than $16
filtered_combinations = grouped_data[grouped_data['eCPM'] > 16]

# Getting all unique sub-verticals with eCPM > $16 for each vertical
sub_verticals_eCPM_over_16 = filtered_combinations.groupby('Vertical')['Sub Vertical'].unique()

# Mapping these sub-verticals back to the 'passed offers' dataframe
passed_offers['Data Vertical Recommendations'] = passed_offers['Vertical'].map(sub_verticals_eCPM_over_16)

In [16]:
passed_offers

,Hitpath Offer ID,Scheduling Name,Advertiser,$ Payout,Payout Type,Vertical,Day Restrictions,Revenue,Delivered,eCPM,Data Vertical Recommendations
0,12792,12792-Shopping-Perpay-Perpay - Holiday,Perpay,$7.00,CPA,Shopping,NaN,91.00,6459.0,14.088868,NaN
1,12588,12588-Loan-EPCVIP-Money Loan USA,EPCVIP,Dynamic,RevShare,Loan,NaN,161.99,7197.0,22.507989,"[Credit Card, Financial, Personal Loan, Rent T..."
2,12587,12587-Loan-EPCVIP-UnitedLoans,EPCVIP,Dynamic,RevShare,Loan,NaN,158.62,8659.0,18.318513,"[Credit Card, Financial, Personal Loan, Rent T..."
3,12710,12710-Resources-What If-Unemployment Benefits ...,What If,$0.55,CPA,Resources,NaN,95.70,4478.0,21.371148,"[Credit Card, Financial, Surveys, Sweepstakes]"


# Last 30 Days of Raw Data

In [17]:
#last 30 days of data minus current and last day

end_date = (dt.date.today() - dt.timedelta(days=1))  # Subtract today
start_date = end_date - dt.timedelta(days=30)

#convert date format
start_date= start_date.strftime('%Y-%m-%d')
end_date = end_date.strftime('%Y-%m-%d')

df_30 = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
df_30['Hitpath Offer ID'] = df_30['Hitpath Offer ID'].astype(str)
df_30['Hitpath Offer ID'] = df['Hitpath Offer ID'].replace(r'\.0$', '', regex=True)
df_30 = df_30[df_30['Send Strategy'] != 'AR']



<ipython-input-17-964be40463c5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_30['Hitpath Offer ID'] = df_30['Hitpath Offer ID'].astype(str)
<ipython-input-17-964be40463c5>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_30['Hitpath Offer ID'] = df['Hitpath Offer ID'].replace(r'\.0$', '', regex=True)


In [18]:
#dataset eCPM last 30 days
eCPM_df = df_30.groupby('shortcode_DP.SV').agg({'Revenue':'sum','Delivered':'sum'}).reset_index()
eCPM_df['eCPM'] = eCPM_df['Revenue'] * 1000 / eCPM_df['Delivered']

# Historical Offer Performance

In [19]:
#total historical offer performance
offer_performance = df.groupby(['shortcode_DP.SV', 'Offer', 'Hitpath Offer ID']).agg({'Revenue':'sum','Delivered':'sum', 'Opportunity Cost' : 'sum'}).reset_index()
offer_performance['Hitpath Offer ID'] = offer_performance['Hitpath Offer ID'].astype(int).astype(str)
offer_performance.rename(columns={'Revenue' : 'Overall Revenue',
                                 'Delivered': 'Overall Delivered'}, inplace=True)

#match shortcode_dp.sv
values_to_match = eCPM_df['shortcode_DP.SV']
offer_performance = offer_performance[offer_performance['shortcode_DP.SV'].isin(values_to_match)]
offer_performance['Overall Offer eCPM'] = offer_performance['Overall Revenue']*1000/offer_performance['Overall Delivered']

In [20]:
#last date drop

# Convert the date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Group by the unique value(s) and find the index of the most recent date
idx = df.groupby(['shortcode_DP.SV', 'Offer', 'Hitpath Offer ID'])['Date'].idxmax()

# Use the index to select the rows with the most recent date
last_drop = df.loc[idx]
last_drop = last_drop[['Affiliate_Id','shortcode_DP.SV', 'Offer', 'Date', 'Delivered', 'Revenue']]
last_drop['Last Drop eCPM'] = last_drop['Revenue'] * 1000 / last_drop['Delivered']

# Merge the dataframes on the 'shortcode_DP.SV' and 'Offer' columns
offer_performance = pd.merge(offer_performance, last_drop, on=['shortcode_DP.SV', 'Offer'], how='left')
offer_performance = offer_performance.rename(columns={
    'Date': 'Last Drop Date',
})

offer_performance.rename(columns={'Delivered': 'Most Recent Delivered Volume', 
                                  'Revenue' : 'Most Recent Offer Revenue'}, inplace=True)

In [21]:
#check if the drop occured in the last 14 days

# Get the current date
current_date = pd.to_datetime('today')

# Calculate the difference in days between the current date and the 'Last Drop Date'
days_difference = (current_date - offer_performance['Last Drop Date']).dt.days

# Add a new column that tells if the date was within the last 14 days
offer_performance['Last 14 Days'] = days_difference <= 14

In [22]:
sorted_offer_performance = offer_performance.sort_values(by=['shortcode_DP.SV', 'Last Drop eCPM', 'Last 14 Days'], 
                                                    ascending=[True, False, True])

#merge statuses
sorted_offer_performance = sorted_offer_performance.merge(sms_offer[['Hitpath Offer ID', 'Status']], 
                                                          how='left', 
                                                          on='Hitpath Offer ID')

#select only production statuses
sorted_offer_performance = sorted_offer_performance[
    sorted_offer_performance['Status'] == 'Live']

In [23]:
#overall dataset eCPM
newdf = sorted_offer_performance.groupby('shortcode_DP.SV').agg({'Overall Revenue':'sum','Overall Delivered':'sum'}).reset_index()
newdf['Overall Dataset eCPM'] =  newdf['Overall Revenue']*1000/newdf['Overall Delivered']
newdf = newdf.drop(columns=['Overall Revenue', 'Overall Delivered'], axis=1)

In [24]:
#merge dataset eCPM
sorted_offer_performance = sorted_offer_performance.merge(newdf, on="shortcode_DP.SV", how="left")

#merge pubid restrictions
sorted_offer_performance = sorted_offer_performance.merge(sms_restrictions, on="Hitpath Offer ID", how="left")

In [25]:
#remove rows if aff id is in pubid restriction
sorted_offer_performance = sorted_offer_performance[~sorted_offer_performance.apply(lambda row: str(row['Affiliate_Id']) in str(row['Paused Pubs']), axis=1)]

#drop aff id and paused pub column
sorted_offer_performance = sorted_offer_performance.drop(columns=['Affiliate_Id','Paused Pubs'], axis = 1)

#drop mismatching ids
sorted_offer_performance['offer_abbreviation'] = sorted_offer_performance['Offer'].str[-3:]

# Extract the 3 letter shortcode abbreviation from the 'shortcode_DP.SV' column
sorted_offer_performance['shortcode_abbreviation'] = sorted_offer_performance['shortcode_DP.SV'].str.split('_').str[0]

# Filter rows where the abbreviations do not match
sorted_offer_performance = sorted_offer_performance[sorted_offer_performance['offer_abbreviation'] == sorted_offer_performance['shortcode_abbreviation']]

# Drop the temporary columns
sorted_offer_performance = sorted_offer_performance.drop(columns=['offer_abbreviation', 'shortcode_abbreviation'])

In [26]:
# #within 70% of overall
# sorted_offer_performance = sorted_offer_performance[sorted_offer_performance['Last Drop eCPM'] >= 0.7 * sorted_offer_performance['Overall Dataset eCPM']]


In [27]:
grouped = sorted_offer_performance.groupby('shortcode_DP.SV')
filtered_dfs = []

for name, group in grouped:
    # Filtering offers with Overall Offer eCPM >= Overall Dataset eCPM
    filtered_group = group[group['Last Drop eCPM'] >= group['Overall Dataset eCPM']]

    # If less than 7 offers, include offers with at least 90% of Overall Dataset eCPM, then 85%, and so on.
    thresholds = [0.9, 0.85, 0.8, 0.75, 0.7]
    for threshold in thresholds:
        if len(filtered_group) < 7:
            additional_offers = group[group['Last Drop eCPM'] >= group['Overall Dataset eCPM'] * threshold]
            filtered_group = pd.concat([filtered_group, additional_offers]).drop_duplicates()
        else:
            break
   
    if len(filtered_group) > 7:
        filtered_group = filtered_group.sort_values(by='Last Drop eCPM', ascending=False).head(7)

    filtered_dfs.append(filtered_group)

# Combining all filtered groups into a single DataFrame
filtered_df = pd.concat(filtered_dfs)

In [28]:
#Opp cost eCPM
filtered_df['Opportunity Cost eCPM'] = filtered_df['Opportunity Cost'
                                                                  ]*1000/filtered_df['Overall Delivered']
filtered_df = filtered_df.drop(columns=['Opportunity Cost'])

In [29]:
#Move Opporunity Cost eCPM Column
column_to_move = filtered_df.pop("Opportunity Cost eCPM")

# insert column with insert(location, column_name, column_value)

filtered_df.insert(5, "Opportunity Cost eCPM", column_to_move)

In [30]:
# BY DMA Account
filtered_df['extracted_DP.SV'] = filtered_df['shortcode_DP.SV'].str.split('_').str[-1]

#merge using 'extracted_DP.SV' and 'DP.SV'
filtered_df = pd.merge(filtered_df, pub_config[['DP.SV', 'DMA']], 
                     left_on='extracted_DP.SV', right_on='DP.SV', how='left')

#drop temp columns
filtered_df = filtered_df.drop(columns=['extracted_DP.SV', 'DP.SV'])

In [31]:
dma_separated_dfs = {}
for dma in filtered_df['DMA'].unique():
    if pd.notna(dma):  # Ensure that the 'DMA' value is not NaN
        dma_separated_dfs[dma] = filtered_df[filtered_df['DMA'] == dma]

# Create Data for Usage in Recommended Drops

In [32]:
#last 60 days of data minus current and last day

end_date = (dt.date.today() - dt.timedelta(days=1))  # Subtract today
start_date = end_date - dt.timedelta(days=30)

#convert date format
start_date= start_date.strftime('%Y-%m-%d')
end_date = end_date.strftime('%Y-%m-%d')

df_60 = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

In [33]:
# match dataset vertical to dp.sv
# df_dv_merged = df_60.merge(pub_config[['DP.SV', 'Sub Vertical', 'INTERNAL STATUS']], on='DP.SV', how='left')
df_dv_merged = df.merge(pub_config[['DP.SV', 'Sub Vertical', 'INTERNAL STATUS']], on='DP.SV', how='left')

# match offer vertical with hitpath ID
df_dvov_merged = df_dv_merged.merge(sms_offer[['Hitpath Offer ID', 'Vertical', 'Paused Pubs', 'Status']], on='Hitpath Offer ID', how='left')

#rename
df_dvov_merged.rename(columns={'Vertical': 'Offer Vertical', 'Sub Vertical' : 'Data Vertical'}, inplace=True)


In [34]:
#only approved offers
df_dvov_merged = df_dvov_merged[df_dvov_merged['Status'] == 'Live']

In [35]:
#active pubs
df_dvov_merged = df_dvov_merged[df_dvov_merged['INTERNAL STATUS'] == 'ACTIVE']

In [36]:
#delivered > 0
df_dvov_merged = df_dvov_merged[df_dvov_merged['Delivered'] > 0]


In [37]:
#remove lead group specific offers
a = ['12384', '12385', '12386']
df_dvov_merged = df_dvov_merged[~df_dvov_merged['Hitpath Offer ID'].isin(a)]


# Offer Vertical OCE By Data Vertical

In [38]:
#find data
data_offer_vert_OCE = df_dvov_merged.groupby(['Data Vertical', 'Offer Vertical']).agg({
    'Revenue':'sum',
    'Delivered':'sum', 
    'Opportunity Cost' : 'sum'}).reset_index()


In [39]:
#calculate Opportunity Cost eCPM
data_offer_vert_OCE['Opportunity Cost eCPM'] = data_offer_vert_OCE['Opportunity Cost'
                                                                  ]*1000/data_offer_vert_OCE['Delivered']

In [40]:
#filter if greater than -1
data_offer_vert_OCE = data_offer_vert_OCE[data_offer_vert_OCE['Opportunity Cost eCPM'] > -5]


In [41]:
data_offer_vert_OCE

,Data Vertical,Offer Vertical,Revenue,Delivered,Opportunity Cost,Opportunity Cost eCPM
0,Benefits,Credit Card,414.0000,25190.0,192.175852,7.629053
1,Benefits,Loan,80.0000,10421.0,-12.635596,-1.212513
2,Benefits,Resources,542.3000,42510.0,171.162040,4.026395
4,Credit Card,Auto Warranty,15244.0000,393272.0,3688.930323,9.380099
5,Credit Card,Cellular,2461.0000,127132.0,-446.437013,-3.511602
7,Credit Card,Credit Score,1448.8000,128785.0,-520.040640,-4.038053
10,Credit Card,Healthcare,508.8000,29302.0,-16.834762,-0.574526
14,Credit Card,Resources,53748.8500,2986152.0,-14731.835936,-4.933384
17,Financial,Auto Warranty,10080.0000,213045.0,1859.927241,8.730208
27,Financial,Resources,76826.4500,2264124.0,-5534.616772,-2.444485


# Offer Opportunity Cost eCPM

In [42]:
#find data 
offer_OCE = df_dvov_merged.groupby(['Hitpath Offer ID','Offer Vertical']).agg({
    'Revenue':'sum',
    'Delivered':'sum', 
    'Opportunity Cost' : 'sum'}).reset_index()


In [43]:
#calculate Opportunity Cost eCPM
offer_OCE['Opportunity Cost eCPM'] = offer_OCE['Opportunity Cost']*1000/offer_OCE['Delivered']

In [44]:
#filter if greater than -1
offer_OCE = offer_OCE[offer_OCE['Opportunity Cost eCPM'] > -5]


In [45]:
offer_OCE

,Hitpath Offer ID,Offer Vertical,Revenue,Delivered,Opportunity Cost,Opportunity Cost eCPM
5,11600,Resources,98875.0000,3357736.0,15138.064022,4.508414
12,11891,Auto Warranty,73734.0000,1909147.0,21371.442645,11.194236
24,12233,Loan,20922.2735,795944.0,1348.107951,1.693722
32,12514,Legal,3816.0000,249829.0,-29.783913,-0.119217
34,12586,Healthcare,1046.4000,73701.0,-109.560779,-1.486558
39,12640,Resources,1913.0000,99011.0,-15.705708,-0.158626
40,12710,Resources,95.7000,4478.0,27.765666,6.200461
45,6763,Rent To Own,11725.0000,1054493.0,-3394.472776,-3.219057
47,8795,Employment,7128.0000,609406.0,219.607546,0.360363
49,9088,Resources,122866.5000,7686232.0,-32042.986206,-4.168881


# Rec

In [46]:
#remove old shortcode_DP.SV
# Combine 'DP.SV' and 'shortcode_DP.SV' to create a unique key
df_dvov_merged['unique_key'] = df_dvov_merged['DP.SV'] + "_" + df_dvov_merged['shortcode_DP.SV']

# Find the latest date for each unique key
latest_dates = df_dvov_merged.groupby('unique_key')['Date'].max()

# Calculate the date 30 days ago from the most recent date in the dataset
date_threshold = df_dvov_merged['Date'].max() - dt.timedelta(days=15)

# Identifiers that have had a drop within the last 30 days
active_keys = latest_dates[latest_dates > date_threshold]

# Filter the original DataFrame to only include rows with these active keys
df_dvov_merged = df_dvov_merged[df_dvov_merged['unique_key'].isin(active_keys.index)]



In [47]:
# Create an empty dataframe for the updated recommendations
recommendations = pd.DataFrame(columns=["shortcode_DP.SV", "PubID", 
                                               "Hitpath Offer ID","Offer Vertical"])

# Iterate over each unique shortcode_DP.SV in df
for shortcode in df_dvov_merged['shortcode_DP.SV'].unique():
    # Filter the dataframe for the specific shortcode
    temp_df = df_dvov_merged[df_dvov_merged['shortcode_DP.SV'] == shortcode]
    
    # Extract the data vertical and PubID
    data_vertical = temp_df['Data Vertical'].iloc[0]
    pub_id = temp_df['Affiliate_Id'].iloc[0]
    
    # Iterate over each Hitpath ID in offer_OCE
    for _, row in offer_OCE.iterrows():
        hitpath_id = row['Hitpath Offer ID']
        offer_vertical = row['Offer Vertical']
        
        # Check if this Hitpath ID has been sent by the current shortcode_DP.SV
        if hitpath_id not in temp_df['Hitpath Offer ID'].values:
            # Check if the shortcode_DP.SV's Data Vertical appears in offer_OCE for this Offer Vertical
            potential_matches = data_offer_vert_OCE[(data_offer_vert_OCE['Data Vertical'] == data_vertical) & 
                                             (data_offer_vert_OCE['Offer Vertical'] == offer_vertical)]
            if len(potential_matches) > 0:
                # Check if the PubID is not in the Paused Pubs column for this Hitpath ID
                paused_pubs = df_dvov_merged[df_dvov_merged['Hitpath Offer ID'] == hitpath_id]['Paused Pubs'].values
                if len(paused_pubs) > 0 and pub_id not in paused_pubs:
                    recommendations = recommendations.append({
                        "shortcode_DP.SV": shortcode,
                        "PubID": pub_id,
                        "Hitpath Offer ID": hitpath_id,
                        "Offer Vertical": offer_vertical
                    }, ignore_index=True)

recommendations

,shortcode_DP.SV,PubID,Hitpath Offer ID,Offer Vertical
0,HZB_AL.PL,461794,12233,Loan
1,HZB_AL.PL,461794,6763,Rent To Own
2,HZB_RHD.CC,461263,12640,Resources
3,HZB_RHD.CC,461263,12710,Resources
4,DSS_RHD.CC,461263,12710,Resources
5,UAA_SPK.CR2,461842,12586,Healthcare
6,UAA_SPK.CR2,461842,12640,Resources
7,UAA_SPK.CR2,461842,12710,Resources
8,MBC_PN.FC,461653,12710,Resources
9,SVT_AL.PL,461794,6763,Rent To Own


In [48]:
#checking for paused pubids
recommendations = recommendations.merge(sms_offer[['Hitpath Offer ID', 'Scheduling Name']], on='Hitpath Offer ID', how='left')
recommendations = recommendations.merge(sms_restrictions, on="Hitpath Offer ID", how="left")

In [49]:
#remove rows if aff id is in pubid restriction
recommendations = recommendations[~recommendations.apply(lambda row: str(row['PubID']) in str(row['Paused Pubs']), axis=1)]

#drop aff id and paused pub column
recommendations = recommendations.drop(columns=['PubID','Paused Pubs'], axis = 1)

In [50]:
# mamba_future['Hitpath Offer ID'] = mamba_future['Hitpath Offer ID'].astype(int).astype(str)

#matching hitpath id and shortcode to find matches in rec tab to future scheduledd
matches = recommendations.apply(
    lambda x: any(
        (mamba_future['Hitpath Offer ID'] == x['Hitpath Offer ID']) & 
        (mamba_future['shortcode_DP.SV'] == x['shortcode_DP.SV'])
    ), 
    axis=1
)

# remove matching rows from recommendations tab
recommendations_final = recommendations[~matches]

recommendations_final

,shortcode_DP.SV,Hitpath Offer ID,Offer Vertical,Scheduling Name
0,HZB_AL.PL,12233,Loan,12233-Loan-Bright Horizon-5kFunds.com
1,HZB_AL.PL,6763,Rent To Own,6763-Rent To Own-Britebox-Rent To Own Research
2,HZB_RHD.CC,12640,Resources,12640-Resources-Pushnami-Found Cash
3,HZB_RHD.CC,12710,Resources,12710-Resources-What If-Unemployment Benefits ...
4,DSS_RHD.CC,12710,Resources,12710-Resources-What If-Unemployment Benefits ...
5,UAA_SPK.CR2,12586,Healthcare,12586-Healthcare-DMS (W4)-Obamacare Rates - AC...
6,UAA_SPK.CR2,12640,Resources,12640-Resources-Pushnami-Found Cash
7,UAA_SPK.CR2,12710,Resources,12710-Resources-What If-Unemployment Benefits ...
8,MBC_PN.FC,12710,Resources,12710-Resources-What If-Unemployment Benefits ...
9,SVT_AL.PL,6763,Rent To Own,6763-Rent To Own-Britebox-Rent To Own Research


In [51]:
#output file
filename = filepath.output_folder+'/Redrop - {}.xlsx'.format(dt.date.today().strftime("%m_%d_%Y"))

#create writer
writer = pd.ExcelWriter(filename, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
with pd.ExcelWriter(filename) as writer:
    for dma, df in dma_separated_dfs.items():
        sheet_name = f'{dma} - Redrop'  # dynamic sheet name based on the DMA
        df.to_excel(writer, sheet_name=sheet_name, index=False)  # Writing data to different sheets

with pd.ExcelWriter(filename, engine='openpyxl', mode='a') as writer:
    recommendations_final.to_excel(writer, sheet_name='Offer Recommendations', index=False)

with pd.ExcelWriter(filename, engine='openpyxl', mode='a') as writer:
    passed_offers.to_excel(writer, sheet_name='Passed Offers - Last 30 days', index=False)

# Close the Pandas Excel writer and output the Excel file.
# writer.close()

In [52]:
toaddr = ['lili@rxmg.com', 'g.chao@rxmg.com','nathan@rxmg.com', 'prasad@rxmg.com']
# toaddr = ['nathan@rxmg.com']
today = dt.date.today().strftime("%m_%d_%Y")
subject_line = f"SMS Team Weekly Redrop/Recommendations Report - {today}"
email_body = "Hi Team, here is the Weekly Redrop Report \n\n"

for i in toaddr:
    send_email.send_email([filename],subject_line,email_body,i)
 